# XML to CSV

To receive some data entries to work with in the database, I use [https://export.arxiv.org](https://export.arxiv.org) via the query endpoint.

The query: [https://export.arxiv.org/api/query?search_query=cat:cs.*&start=0&max_results=300&sortBy=submittedDate&sortOrder=descending](https://export.arxiv.org/api/query?search_query=cat:cs.*&start=0&max_results=300&sortBy=submittedDate&sortOrder=descending
).

In [64]:
import os, ssl
import urllib, urllib.request

# Ignores certificate verification for now
ctx = ssl._create_unverified_context()

url = "https://export.arxiv.org/api/query?search_query=cat:cs.*&start=0&max_results=200&sortBy=submittedDate&sortOrder=descending"
data = urllib.request.urlopen(url, context=ctx)
content = data.read().decode('utf-8')

XML_PATH = 'arxiv_papers.xml'

if os.path.exists(XML_PATH):
    os.remove(XML_PATH)

with open(XML_PATH, 'w') as file:
    file.write(content)

In [65]:
import xml.etree.ElementTree as ET
import csv

XML_PATH = 'arxiv_papers.xml'
CSV_PATH = 'arxiv_papers.csv'

tree = ET.parse(XML_PATH)
root = tree.getroot()

if os.path.exists(CSV_PATH):
    os.remove(CSV_PATH)

with open(CSV_PATH, 'w', newline='') as csvfile:
    csvwriter = csv.writer(csvfile)
    
    csvwriter.writerow(['title', 'author', 'published'])
    
    for entry in root.findall('{http://www.w3.org/2005/Atom}entry'):
        title = entry.find('{http://www.w3.org/2005/Atom}title').text
        author = ', '.join([author.find('{http://www.w3.org/2005/Atom}name').text for author in entry.findall('{http://www.w3.org/2005/Atom}author')])
        published = entry.find('{http://www.w3.org/2005/Atom}published').text
        
        csvwriter.writerow([title, author, published])